In [ ]:
import os
from lib.feeders import RNNFeeder, MLPFeeder
from lib.decoders import LanguageModel, Decoder
from lib.models import BaseModel

%load_ext autoreload
%autoreload 2

In [ ]:
feeder = MLPFeeder("../data/features/SkodaAuto_25_10_log_filterbank_energies.hdf5")

feeder.remove_tmp_storage()
feeder.create_datasets((0.75, 0.5), left_context=10, right_context=5, sample=[3, 3, 3])

In [ ]:
feeder = RNNFeeder("../data/features/SkodaAuto_25_10_log_filterbank_energies.hdf5")

feeder.remove_tmp_storage()
feeder.create_datasets((0.75, 0.5), 10, left_context=0, right_context=0, sample=[5, 5, 2])

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout


class Model(BaseModel):
    def __init__(self, *args, **kwargs):
        super(Model, self).__init__(*args, **kwargs)

    def _compile_model(self):
        model = Sequential()
        model.add(Dense(units=1028, input_dim=self.input_shape, activation="relu"))
        model.add(Dropout(0.5))
        model.add(Dense(units=64, input_dim=1028, activation="relu"))
        model.add(Dropout(0.5))
        model.add(Dense(units=1028, input_dim=64, activation="relu"))
        model.add(Dropout(0.5))

        model.add(Dense(units=self.output_shape, activation="softmax"))
        
        model.compile(loss="categorical_crossentropy", optimizer="sgd", metrics=["accuracy"])

        return model

In [ ]:
from keras.models import Sequential
from keras.layers import GRU, Dense


class Model(BaseModel):
    def __init__(self, *args, **kwargs):
        super(Model, self).__init__(*args, **kwargs)

    def _compile_model(self):
        model = Sequential()
        model.add(GRU(128, dropout=0.25, recurrent_dropout=0.25, return_sequences=True, input_shape=self.input_shape))
        model.add(GRU(128, dropout=0.25, recurrent_dropout=0.25))
        model.add(Dense(self.output_shape, activation="softmax"))

        model.compile(loss="categorical_crossentropy", optimizer="rmsprop", metrics=["accuracy"])

        return model

In [ ]:
model = Model(feeder, 2, 32, ["modelCheckpoint", "tensorboard", "batchPrint", "reduceLROnPlateau"])
#model.load_weights()
model.train()
predictions, transcriptions = model.predict()
model.evaluate()

In [ ]:
features_name = os.path.splitext(os.path.basename(feeder.features_path))[0]
model_name = model.__class__.__name__

results_dir = os.path.join("..", "results", features_name, model_name)

if not os.path.exists(results_dir):
    os.makedirs(results_dir)

pred_path = os.path.join(results_dir, "pred.mlf")
ref_path = os.path.join(results_dir, "ref.mlf")
phonemes_path = os.path.join(results_dir, "phonemes")

In [ ]:
ngram = 2
is_bigram = ngram == 2

languageModel = LanguageModel(feeder)
language_model = languageModel.create_model(ngram=ngram)

decoder = Decoder(feeder, language_model, is_bigram)

with open(pred_path, "w") as fw_pred, open(ref_path, "w") as fw_ref:
    fw_pred.write("#!MLF!#\n")
    fw_ref.write("#!MLF!#\n")

    for i, (observations, transcription) in enumerate(zip(predictions, transcriptions)):
        decoded_transcription = decoder.decode(observations)

        fw_pred.write('"*/{}.rec"\n'.format(i))
        fw_ref.write('"*/{}.lab"\n'.format(i))

        fw_pred.write("\n".join(decoded_transcription))
        fw_ref.write("\n".join(transcription))

        fw_pred.write("\n.\n")
        fw_ref.write("\n.\n")
        
        print("Decoded:")
        print(decoded_transcription)
        print()
        print("Ground Truth:")
        print(list(transcription))
        print()

with open(phonemes_path, "w") as fw:
    fw.write("\n".join(feeder.encoder.classes_))

In [ ]:
from subprocess import check_output

command = "HResults -I {} {} {}".format(ref_path, phonemes_path, pred_path)

for line in check_output(command.split(" ")).decode().split("\n"):
    print(line)

In [ ]:
feeder.remove_tmp_storage()